In [1]:
import pandas as pd
import neo4j as neo

In [40]:
spotify = pd.read_csv('spotify/spotify.csv')

In [41]:
spotify = spotify.sample(2000)

In [42]:
feats = ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

for feat in feats:
    # Standardize the feature
    spotify[feat] = (spotify[feat] - spotify[feat].mean())/spotify[feat].std() 

In [43]:
spotify = dict(spotify.T)

In [ ]:
password = 'j4f^q3WrwZYw7u3gx&JsgknBnTkUJPX'

# Connect to neo4j
neo4j = neo.GraphDatabase.driver('neo4j://localhost:7687', auth=('neo4j', password))


def cosine_similarity(v1, v2):
    return sum([a*b for a,b in zip(v1, v2)]) / (sum([a**2 for a in v1])**0.5 * sum([b**2 for b in v2])**0.5)

with neo4j.session() as session:
    session.run('MATCH (n) DETACH DELETE n')
    for i, track in spotify.items():
        session.run('CREATE (t:Track {name: $name, artists: $artists, genre: $genre, popularity: $popularity, duration: $duration, explicit: $explicit, danceability: $danceability, energy: $energy, key: $key, loudness: $loudness, mode: $mode, speechiness: $speechiness, acousticness: $acousticness, instrumentalness: $instrumentalness, liveness: $liveness, valence: $valence, tempo: $tempo, time_signature: $time_signature})', name=track['track_name'], artists=track['artists'], genre=track['track_genre'], popularity=track['popularity'], duration=track['duration_ms'], explicit=track['explicit'], danceability=track['danceability'], energy=track['energy'], key=track['key'], loudness=track['loudness'], mode=track['mode'], speechiness=track['speechiness'], acousticness=track['acousticness'], instrumentalness=track['instrumentalness'], liveness=track['liveness'], valence=track['valence'], tempo=track['tempo'], time_signature=track['time_signature'])
        for j, other_track in spotify.items():
            if i != j:
                similarity = cosine_similarity([track['danceability'], track['energy'], track['key'], track['loudness'], track['mode'], track['speechiness'], track['acousticness'], track['instrumentalness'], track['liveness'], track['valence'], track['tempo'], track['time_signature']], [other_track['danceability'], other_track['energy'], other_track['key'], other_track['loudness'], other_track['mode'], other_track['speechiness'], other_track['acousticness'], other_track['instrumentalness'], other_track['liveness'], other_track['valence'], other_track['tempo'], other_track['time_signature']])
                if similarity > 0.4:
                    session.run('MATCH (t1:Track {name: $name1}), (t2:Track {name: $name2}) CREATE (t1)-[:SIMILAR_TO {similarity: $similarity}]->(t2)', name1=track['track_name'], name2=other_track['track_name'], similarity=similarity)